# Developing Alternatives to VaR (Value-at-Risk)

| Name            | Email                          |
|-----------------|--------------------------------|
| Rohit Singh     | rohitkumar.singh7885@gmail.com |
| Sahil Shinde    | sahil311292@gmail.com          |
| Shantanu Mishra | 8hantanu@gmail.com             |

## Pre-requisites

### Install packages

In [1]:
!pip install yfinance
!pip install pandas_datareader

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.3/60.3 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 7.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 11.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 15.3 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 502.3/502.3 kB 8.8 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 14.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.0/143.0 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.2/112.2 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.2/179.2 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.8/341.8 kB 7.3 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━

### Imports

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pandas_datareader import data as pdr
import yfinance as yf
yf.pdr_override()

## Import data 
Data from year 2000 to 2020 for treasury yields for 1, 5, and 10 years

### US treasury bonds rates for 1 year yields

In [94]:
treasury_1y = pdr.get_data_yahoo('^IRX', start='2010-01-01', end='2020-12-31')
treasury_1y.head()

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2010-01-04,0.080,0.080,0.055,0.055,0.055,0
2010-01-05,0.060,0.065,0.040,0.060,0.060,0
2010-01-06,0.060,0.060,0.045,0.045,0.045,0
2010-01-07,0.045,0.055,0.040,0.045,0.045,0
2010-01-08,0.045,0.045,0.035,0.040,0.040,0


### US treasury bonds rates for 5 years yields

In [95]:
treasury_5y = pdr.get_data_yahoo('^FVX', start='2010-01-01', end='2020-12-31')
treasury_5y.head()

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2010-01-04,2.684,2.684,2.633,2.652,2.652,0
2010-01-05,2.583,2.593,2.549,2.558,2.558,0
2010-01-06,2.566,2.625,2.558,2.573,2.573,0
2010-01-07,2.625,2.642,2.574,2.600,2.600,0
2010-01-08,2.642,2.654,2.527,2.566,2.566,0


### US treasury bonds rates for 10 years yields

In [96]:
treasury_10y = pdr.get_data_yahoo('^TNX', start='2010-01-01', end='2020-12-31')
treasury_10y.head()


[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2010-01-04,3.859,3.859,3.808,3.841,3.841,0
2010-01-05,3.790,3.800,3.749,3.755,3.755,0
2010-01-06,3.771,3.837,3.761,3.808,3.808,0
2010-01-07,3.845,3.859,3.800,3.822,3.822,0
2010-01-08,3.843,3.851,3.775,3.808,3.808,0


In [62]:
# Gather equity data from 2002-2022
sp500 = pdr.DataReader(['^GSPC'], start = '2002-01-01', end = '2022-12-31')
sp500.to_csv("/Users/sahil/Study/MScFE/Capstone Project 2/mscfe_690/Data/sp500.csv")
sp500.head()


[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2002-01-02,1148.079956,1154.670044,1136.229980,1154.670044,1154.670044,1171000000
2002-01-03,1154.670044,1165.270020,1154.010010,1165.270020,1165.270020,1398900000
2002-01-04,1165.270020,1176.550049,1163.420044,1172.510010,1172.510010,1513000000
2002-01-07,1172.510010,1176.969971,1163.550049,1164.890015,1164.890015,1308300000
2002-01-08,1164.890015,1167.599976,1157.459961,1160.709961,1160.709961,1258800000


In [64]:
apple = pdr.DataReader(['AAPL'], start = '2002-01-01', end = '2022-12-31')
apple.to_csv("/Users/sahil/Study/MScFE/Capstone Project 2/mscfe_690/Data/apple.csv")
apple.head()


[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2002-01-02,0.393750,0.416071,0.392143,0.416071,0.354126,529496800
2002-01-03,0.410714,0.424107,0.406607,0.421071,0.358381,612007200
2002-01-04,0.416786,0.427679,0.410536,0.423036,0.360054,409976000
2002-01-07,0.423571,0.428571,0.406250,0.408929,0.348047,444584000
2002-01-08,0.406250,0.411607,0.401071,0.403750,0.343639,450038400


In [65]:
microsoft = pdr.DataReader(['MSFT'], start = '2002-01-01', end = '2022-12-31')
microsoft.to_csv("/Users/sahil/Study/MScFE/Capstone Project 2/mscfe_690/Data/microsoft.csv")
microsoft.head()

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2002-01-02,33.325001,33.555000,32.755001,33.520000,20.913074,48124000
2002-01-03,33.555000,34.625000,33.544998,34.615002,21.596237,67590800
2002-01-04,34.625000,34.955002,34.334999,34.450001,21.493301,52731400
2002-01-07,34.875000,35.009998,34.174999,34.279999,21.387236,58508800
2002-01-08,34.345001,34.930000,34.000000,34.689999,21.643036,47736400


In [66]:
sgx = pdr.DataReader(['S68.SI'], start = '2002-01-01', end = '2022-12-31')
sgx.to_csv("/Users/sahil/Study/MScFE/Capstone Project 2/mscfe_690/Data/sgx.csv")
sgx.head()

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2002-01-01,1.24,1.24,1.24,1.24,0.409837,0
2002-01-02,1.23,1.23,1.15,1.21,0.399921,8008000
2002-01-03,1.21,1.21,1.21,1.21,0.399921,0
2002-01-04,1.26,1.27,1.24,1.25,0.413142,5809000
2002-01-07,1.26,1.28,1.26,1.27,0.419752,4219000


In [67]:
dollartree = pdr.DataReader(['DLTR'], start = '2002-01-01', end = '2022-12-31')
dollartree.to_csv("/Users/sahil/Study/MScFE/Capstone Project 2/mscfe_690/Data/dollartree.csv")
dollartree.head()

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2002-01-02,10.333333,10.426667,10.080000,10.200000,10.200000,1978200
2002-01-03,10.196667,10.246667,9.950000,10.116667,10.116667,3203700
2002-01-04,10.146667,10.246667,9.990000,10.066667,10.066667,4050900
2002-01-07,10.000000,10.400000,9.643333,9.913333,9.913333,5038200
2002-01-08,9.953333,9.963333,9.476667,9.533333,9.533333,6130500


### Calculate the daily returns

In [97]:
# Calculate daily returns for treasury yields
treasury_1y['daily_return'] = treasury_1y['Adj Close'].pct_change()
treasury_5y['daily_return'] = treasury_5y['Adj Close'].pct_change()
treasury_10y['daily_return'] = treasury_10y['Adj Close'].pct_change()

# Drop missing values
treasury_1y.dropna(inplace=True)
treasury_5y.dropna(inplace=True)
treasury_10y.dropna(inplace=True)



## Metrics

### Value at Risk (VaR)

In [98]:
def calculate_var(df, confidence_level=0.99):
    """
    Calculate Value at Risk (VaR) for a given confidence level and time horizon
    """
    
    sorted_returns = df.sort_values('daily_return')
    var = -sorted_returns['daily_return'].quantile(1-confidence_level)

    return var

# Calculate VaR round for 1 year treasury yield with 99% confidence level
var_1y = calculate_var(treasury_1y, 0.99)
print(f'Value at Risk for 1 year treasury yield with 99% confidence level is {var_1y:.3f}')

# Calculate VaR for 5 year treasury yield with 99% confidence level
var_5y = calculate_var(treasury_5y, 0.99)
print(f'Value at Risk for 5 year treasury yield with 99% confidence level is {var_5y:.3f}')

# Calculate VaR for 10 year treasury yield with 99% confidence level
var_10y = calculate_var(treasury_10y, 0.99)
print(f'Value at Risk for 10 year treasury yield with 99% confidence level is {var_10y:.3f}')


Value at Risk for 1 year treasury yield with 99% confidence level is 0.698
Value at Risk for 5 year treasury yield with 99% confidence level is 0.105
Value at Risk for 10 year treasury yield with 99% confidence level is 0.071


### Expected Shortfall (ES)

In [99]:
# Compute ES for a 10 day liquidity horizon based on Basel IV norms which uses a 97.5% confidence level
def calculate_es(df, confidence_level=0.975, time_horizon=10):
    """
    Calculate Expected Shortfall (ES) for a given confidence level and time horizon
    """
    
    sorted_returns = df.sort_values('daily_return')
    var = sorted_returns['daily_return'].quantile(1-confidence_level)
    es = -sorted_returns[sorted_returns['daily_return'] <= var]['daily_return'].mean()
    
    return es

# Calculate ES for 1 year treasury yield with 97.5% confidence level
es_1y = calculate_es(treasury_1y, 0.975)
print(f'Expected Shortfall for 1 year treasury yield with 97.5% confidence level is {es_1y:.3f}')

# Calculate ES for 5 year treasury yield with 97.5% confidence level
es_5y = calculate_es(treasury_5y, 0.975)
print(f'Expected Shortfall for 5 year treasury yield with 97.5% confidence level is {es_5y:.3f}')

# Calculate ES for 10 year treasury yield with 97.5% confidence level
es_10y = calculate_es(treasury_10y, 0.975)
print(f'Expected Shortfall for 10 year treasury yield with 97.5% confidence level is {es_10y:.3f}')


Expected Shortfall for 1 year treasury yield with 97.5% confidence level is 0.765
Expected Shortfall for 5 year treasury yield with 97.5% confidence level is 0.113
Expected Shortfall for 10 year treasury yield with 97.5% confidence level is 0.082


### ES based on Differentiated Liquidity Horizon (ESD)

In [100]:
def calculate_es_with_liquidity_horizon(df, confidence_level=0.975):

    time_horizon = [10, 20, 40, 60, 120]
    es_arr = []
    es_scaled_arr = []

    for i, horizon in enumerate(time_horizon):
        es = calculate_es(df, confidence_level, horizon)
        if i != 0:
            es_scaled = es * np.sqrt((time_horizon[i] - time_horizon[i-1]) / 10)
        else:
            es_scaled = es
        es_arr.append(es)
        es_scaled_arr.append(es_scaled)

    es_b = np.sqrt(np.sum(np.square(es_scaled_arr)))

    return es_b

# Calculate ESD for 1 year treasury yield with 97.5% confidence level
esd_1y = calculate_es_with_liquidity_horizon(treasury_1y, 0.975)
print(f'Expected Shortfall for 1 year treasury yield with 97.5% confidence level is {esd_1y:.3f}')

# Calculate ESD for 5 year treasury yield with 97.5% confidence level
esd_5y = calculate_es_with_liquidity_horizon(treasury_5y, 0.975)
print(f'Expected Shortfall for 5 year treasury yield with 97.5% confidence level is {esd_5y:.3f}')

# Calculate ESD for 10 year treasury yield with 97.5% confidence level
esd_10y = calculate_es_with_liquidity_horizon(treasury_10y, 0.975)
print(f'Expected Shortfall for 10 year treasury yield with 97.5% confidence level is {esd_10y:.3f}')


Expected Shortfall for 1 year treasury yield with 97.5% confidence level is 2.650
Expected Shortfall for 5 year treasury yield with 97.5% confidence level is 0.393
Expected Shortfall for 10 year treasury yield with 97.5% confidence level is 0.285


### ES based on Spectral Risk Measure (ESS)

In [101]:
# TODO: